In [ ]:
import os
import getpass
from langchain_community.document_loaders import CSVLoader
from langchain.chat_models import init_chat_model
from langchain_community.embeddings import BaichuanTextEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
# embedding model
os.environ["BAICHUAN_API_KEY"] = ""
# llm model
os.environ["DEEPSEEK_API_KEY"] = ""

In [ ]:
# raw docs
loader = CSVLoader(file_path='./jobs.csv', autodetect_encoding=True, csv_args={
        "delimiter": ","})

docs = []
async for doc in loader.alazy_load():
    docs.append(doc)
print(docs[0].page_content[:300])
print(docs[0].metadata)

company_name: 深圳深度商智科技
job_title: AI智能体工程师
base: 深圳
job_desc: AI智能体工程师招聘岗位说明
 
岗位职责
 
1.?智能体设计与开发：负责AI智能体的整体架构设计与核心算法开发，根据不同的业务场景和需求，运用机器学习、强化学习等技术，打造具备自主决策、任务执行能力的智能体。例如，开发用于客户服务场景的智能聊天体，能理解客户问题并精准回应；或是构建智能物流调度体，优化物流配送路径。
2.?模型训练与优化：收集和整理相关数据，运用大数据处理技术和工具进行数据预处理，使用深度学习框架（如TensorFlow、PyTorch）对智能体模型
{'source': './jobs.csv', 'row': 0}


In [21]:
len(docs)

12

In [ ]:
# chunk
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

In [ ]:
# embed docs
embeddings = BaichuanTextEmbeddings()
vector_store = InMemoryVectorStore(embeddings)
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['b2aadbec-87c3-4716-b73d-65608cec98fc', '1b6897fd-5d8a-4aa0-b007-36a4c6e9f186', 'c238f5c3-7053-4a15-a9b8-fa8e75cb6d4a']


In [ ]:
# DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")

# from openai import OpenAI
 
# # 初始化DeepSeek的API客户端
# client = OpenAI(api_key=DeepSeek_API_KEY, base_url="https://api.deepseek.com")
 
# # 调用DeepSeek的API，生成回答
# response = client.chat.completions.create(
#     model="deepseek-chat",
#     messages=[
#         {"role": "system", "content": "你是乐于助人的助手，请根据用户的问题给出回答"},
#         {"role": "user", "content": "你好，请你介绍一下你自己。"},
#     ],
# )
 
# # 打印模型最终的响应结果
# print(response.choices[0].message.content)

你好！我是一个智能助手，专注于提供信息、解答问题、协助完成任务和提供建议。我可以帮助你处理各种问题，比如学习、工作、生活、技术、娱乐等方面。无论是需要查找资料、写作、翻译、计算，还是需要创意灵感或日常建议，我都可以尽力提供帮助。

我没有个人情感或主观意识，所有的回答都基于已有的数据和算法。如果你有任何问题或需求，随时告诉我，我会尽力满足你的要求！ 😊


In [45]:
model = init_chat_model(model="deepseek-chat", model_provider="deepseek")  

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt", api_url="https://api.smith.langchain.com")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

d:\miniconda3\envs\testenv\lib\site-packages\langsmith\client.py:290: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [ ]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [43]:
result = graph.invoke({"question": "我想找有关大模型agent工程师的全职工作，请帮我找出相关招聘信息，并标注招聘信息来源"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Context: [Document(id='f30da52d-ed39-40d9-8ecb-a14ba8d8ca22', metadata={'source': './jobs.csv', 'row': 8, 'start_index': 0}, page_content='\ufeffcompany_name: 高德\njob_title: 高德-大模型应用开发工程师-Agent方向\nbase: 北京\njob_desc: 职位描述\n团队介绍：\n交通智能团队正在寻找AI造梦师！加入极客天团，用代码重塑未来出行\n团队故事：\n我们是高德地图的的硬核极客天团，过去四年连续解锁交通科技树：\n? 2021年 全球首个分钟级交通事件检测引擎（让堵车预警快过刷朋友圈）\n? 2022年 红绿灯倒计时黑科技（拯救千万路怒症的神级操作）\n? 2023年 V2X车车对话系统（让汽车学会"社牛"沟通）\n? 2024年 AI领航红绿灯（比驾校教练更懂路的老司机）\n现在，我们正在构建智能交通宇宙，等你来编写核心算法剧本！\n\n岗位职责：\n1.负责开发模块化agent框架，整合交通动态数据（轨迹数据、UGC图文、行业信息等），实现多轮交互、实时决策与多智能体协作，为用户智慧出行提供决策支持；\n2.构建和优化分布式推理引擎，确保高并发下的稳定性和低延迟；\n3.负责开发对话记忆管理系统，实现对话历史的智能存储与上下文关联分析，提升多轮交互的语义连贯性。'), Document(id='823726fc-dfb0-4a53-8209-b1baba3d5c96', metadata={'source': './jobs.csv', 'row': 5, 'start_index': 0}, page_content='\ufeffcompany_name: 追觅科技\njob_title: 深度学习C++部署工程师\nbase: 苏州\njob_desc: 1. 负责深度学习模型在端侧设备上的部署与优化，特别是在NPU（神经处理单元）上的部署优化。\n2.从事CV模型在不同平台上的工程化设计、实现与加速、系统集成与工程化落地；\n3.分析并优化模型的推理速度和精度\n4. 与跨职能团队合作，理解系统需求，提供技术支持和解决方案。\n\n任职要求：\n1、擅长c/c++编程，

In [44]:
result = graph.invoke({"question": "我有一些web前端开发经历，想找一份全职工作，请帮我推荐一些招聘信息，并标注招聘信息来源"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Context: [Document(id='6fd56418-efa9-4dc0-92ed-17e74541e54d', metadata={'source': './jobs.csv', 'row': 10, 'start_index': 0}, page_content='\ufeffcompany_name: TikTok\njob_title: 前端开发工程师-TikTok泛架构\nbase: 深圳\njob_desc: 职位描述\n团队介绍：TikTok是一个覆盖150个国家和地区的国际短视频平台，我们希望通过TikTok发现真实、有趣的瞬间，让生活更美好。TikTok 在全球各地设有办公室，全球总部位于洛杉矶和新加坡，办公地点还包括纽约、伦敦、都柏林、巴黎、柏林、迪拜、雅加达、首尔和东京等多个城市。\n\nTikTok泛架构团队处于业务和基础架构之间，我们的目标是拥抱AI，构建最先进的业务架构基建，建立公众信任和合规，关注资源管理和成本优化，用户体验和稳定性，提高工程质量和研发效率。在这里你既能感受到技术对上层业务的赋能，又可以提高在架构层面技术深度的理解，期待你的加入！\n\n1、参与TikTok业务产品研发，深度参与产品需求讨论，C端和B端的功能构建等；\n2、参与设计基于前端和全栈的解决方案，核心代码开发、组件抽象以及系统调优等；\n3、参与系统工程效率和性能优化，提升研发效率以及用户体验；\n4、调研和学习前端新技术，进行技术分享并在重要项目内落地，参与部门前端和全栈技术建设。\n职位要求\n1、2026届获得本科及以上学历，计算机、软件工程等相关专业优先；\n2、热爱计算机科学，对数据分析、前端架构、渲染和性能等技术有热情；\n3、精通JavaScript、HTML、CSS，熟悉DOM、HTTP协议等前端领域知识，至少掌握一种主流前端框架（如React/Vue等）；\n4、有优秀的逻辑分析能力，能够对业务逻辑进行合理的抽象和拆分；\n5、有强烈的求知欲，优秀的学习能力和沟通能力。'), Document(id='823726fc-dfb0-4a53-8209-b1baba3d5c96', metadata={'source': './jobs.csv', 'row': 5, 'start_index': 0}, page_content='\